# Clustering with Spark

## Assignment 9

The following code initializes a Spark session that you use for the remainder of the assignment. 

In [10]:
from pyspark.sql import SparkSession
from pathlib import Path

spark = SparkSession \
    .builder \
    .appName("DSC 400 Assignment 9") \
    .getOrCreate()

# TODO: Change these to point to versions on your local path

sample_kmeans_data_path = 'sample_kmeans_data.txt'
sample_lda_data_path = 'sample_lda_libsvm_data.txt'
sample_movielens_data_path = 'sample_movielens_ratings.txt'

#or load the file using the path
#sample_movielens_data_path = Path('sample_movielens_ratings.txt')
#sample_movielens_data_path.exists() #check if the file exist

### Assignment 9.1

Run the PySpark version of the k-means clustering example found in [Apache Spark's k-means clustering documentation](https://spark.apache.org/docs/latest/ml-clustering.html#k-means). You can also find the code in [Apache Spark's Github repository](https://github.com/apache/spark/tree/master/examples/src/main/python/mllib). 

The example references a `sample_kmeans_data.txt` file. You can find the file at https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_kmeans_data.txt. Put this file in your working path and change the following code to point to your local version of the file. 

```
dataset = spark.read.format("libsvm").load("data/mllib/sample_kmeans_data.txt")
```

In [16]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

dataset = spark.read.format("libsvm").load(sample_kmeans_data_path)

#if the file was loaded using the path lib, then the above code should look as below
#dataset = spark.read.format("libsvm").load(str(sample_kmeans_data_path)) #converted the file into a string

# TODO: Implement the remainder of the code from the k-means clustering example
# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

23/03/03 06:42:51 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
Silhouette with squared euclidean distance = 0.9997530305375207
Cluster Centers: 
[9.1 9.1 9.1]
[0.1 0.1 0.1]


### Assignment 9.2

Run the PySpark version of the Latent Dirichlet Allocation (LDA) example found in [Apache Spark's LDA documentation](https://spark.apache.org/docs/latest/ml-clustering.html#latent-dirichlet-allocation-lda). You can also find the code in [Apache Spark's Github repository](https://github.com/apache/spark/tree/master/examples/src/main/python/mllib). 

The example references a `sample_lda_libsvm_data.txt
` file. You can find the file at https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_lda_libsvm_data.txt
. Put this file in your working path and change the following code to point to your local version of the file. 

```
dataset = dataset = spark.read.format("libsvm").load("data/mllib/sample_lda_libsvm_data.txt")
```

In [17]:
from pyspark.ml.clustering import LDA

# Loads data.
dataset = spark.read.format("libsvm").load(sample_lda_data_path)

# TODO: Implement the remainder of the code from the LDA example
# Trains a LDA model.
lda = LDA(k=10, maxIter=10)
model = lda.fit(dataset)

ll = model.logLikelihood(dataset)
lp = model.logPerplexity(dataset)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

# Describe topics.
topics = model.describeTopics(3)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

# Shows the result
transformed = model.transform(dataset)
transformed.show(truncate=False)

23/03/03 06:45:46 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
23/03/03 06:45:49 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/03 06:45:49 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
The lower bound on the log likelihood of the entire corpus: -795.0318110468672
The upper bound on perplexity: 3.0578146578725662
The topics described by their top-weighted terms:
+-----+-----------+---------------------------------------------------------------+
|topic|termIndices|termWeights                                                    |
+-----+-----------+---------------------------------------------------------------+
|0    |[9, 1, 4]  |[0.10398261025612052, 0.10189617618739767, 0.09803741465748232]|
|1    |[10, 5, 1] |[0.1006969891

### Assignment 9.3

Run the PySpark version of the collaborative filtering example found in [Apache Spark's collaborative filtering documentation](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html). You can also find the code in [Apache Spark's Github repository](https://github.com/apache/spark/tree/master/examples/src/main/python/mllib). 

The example references a `sample_movielens_ratings.txt` file. You can find the file at https://raw.githubusercontent.com/apache/spark/master/data/mllib/als/sample_movielens_ratings.txt. Put this file in your working path and change the following code to point to your local version of the file. 

```
lines = spark.read.text("data/mllib/als/sample_movielens_ratings.txt").rdd
```

In [18]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

lines = spark.read.text(sample_movielens_data_path).rdd

# TODO: Implement the remainder of the code from the collaborative filtering example
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(10)
# Generate top 10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)

# Generate top 10 movie recommendations for a specified set of users
users = ratings.select(als.getUserCol()).distinct().limit(3)
userSubsetRecs = model.recommendForUserSubset(users, 10)
# Generate top 10 user recommendations for a specified set of movies
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies, 10)

Root-mean-square error = 1.6304101084761125


### 